# Image Offsets: Create offset arrays used by drizzle

The offset tables created by a previous notebook (Offsets_2) are used to generate the X and Y offset arrays used by drizzle.

Each offset table will generate 2 arrays, for X and Y respectively, stored as FITS image of short float type.

The algorithms were developed in the Timing notebook; here, they are cast as callable functions used in a loop to process all images in the sequence.

In [1]:
import os, glob
import numpy as np

from astropy.table import Table

import rawpy

In [2]:
datadir = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'

## Functions

In [3]:
# Comparison functions
gt_zero = lambda x: x > 0.0
lt_zero = lambda x: x < 0.0

# Gets the index of the closest star in table. 
# The differences are in the sense pixel - star centroid.
# The comparison functions define from which quadrant the star is drawn from.
def closest(diff_x, diff_y, compare_x, compare_y):
    # Compute mask that masks out everything that is outside 
    # the quadrant defined by the comparison functions
    mask_x = np.where(compare_x(diff_x), 1, 0)
    mask_y = np.where(compare_y(diff_y), 1, 0)
    mask = mask_x * mask_y

    # Get index of star at minimum distance
    distance = np.sqrt((diff_x * diff_x + diff_y * diff_y)) * mask
    if np.nonzero(distance)[0].size > 0:
        mindist = np.min(distance[np.nonzero(distance)])
        index = np.where(distance == mindist )[0][0]
        return index, mindist
    else:
        return -1, 0.0

## Read last table in sequence, and prototype image

Starting with the last table ensures that we get always the same stars along the entire sequence. Offset tables at the beginning of the sequence may include stars that are dropped later on.

In [4]:
# last table in sequence
table_list = glob.glob(datadir + '/*.offsets_table.fits')
table_list.sort()
last = table_list[-1]
offsets_table = Table.read(last)

In [5]:
# prototype image (used to define array size)
image_name = last.split('/')[-1]
image_name = image_name.replace('.offsets_table.fits', '.ARW')
image_name = os.path.join(datadir, image_name)
raw = rawpy.imread(image_name)
imarray = raw.raw_image_visible.astype(float)

ny = imarray.shape[0]
nx = imarray.shape[1]

## Create and populate arrays with interpolated offsets

In [6]:
offset_array_x = np.asarray(imarray)
offset_array_y = np.asarray(imarray)

In [7]:
centroid_x = offsets_table['xcentroid'].data
centroid_y = offsets_table['ycentroid'].data
offset_x = offsets_table['xoffset'].data
offset_y = offsets_table['yoffset'].data

In [10]:
for i in range(0, nx, 2):
    for j in range(0, ny, 2):
        pixel_x = i
        pixel_y = j

        diff_x = pixel_x - centroid_x
        diff_y = pixel_y - centroid_y
        
        index = np.array(range(4), dtype=int)
        dist  = np.array(range(4), dtype=float)
        
        # get index and distance of the closest star, one per quadrant
        index[0], dist[0] = closest(diff_x, diff_y, gt_zero, gt_zero)
        index[1], dist[1] = closest(diff_x, diff_y, lt_zero, gt_zero)
        index[2], dist[2] = closest(diff_x, diff_y, gt_zero, lt_zero)
        index[3], dist[3] = closest(diff_x, diff_y, lt_zero, lt_zero)
        
        # need a cleanup here. Negative indices, zeroed distances.

        # weighted average of the offset values. The weight is the inverse distance pixel-star.
        sumweights = 1./dist[0] + 1./dist[1] + 1./dist[2] + 1./dist[3]
        weighted_offset_x = offset_x[index[0]] * (1./dist[0] / sumweights) + \
                            offset_x[index[1]] * (1./dist[1] / sumweights) + \
                            offset_x[index[2]] * (1./dist[2] / sumweights) + \
                            offset_x[index[3]] * (1./dist[3] / sumweights)
        weighted_offset_y = offset_y[index[0]] * (1./dist[0] / sumweights) + \
                            offset_y[index[1]] * (1./dist[1] / sumweights) + \
                            offset_y[index[2]] * (1./dist[2] / sumweights) + \
                            offset_y[index[3]] * (1./dist[3] / sumweights)

        offset_array_x[i] = weighted_offset_x
        offset_array_y[i] = weighted_offset_y

        print(i, j, index, dist)


        



/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


0 0 [-1 -1 -1  6] [   0.            0.            0.         1208.75703097]
0 2 [-1 -1 -1  6] [   0.            0.            0.         1208.56960145]
0 4 [-1 -1 -1  6] [   0.            0.            0.         1208.38545306]
0 6 [-1 -1 -1  6] [   0.            0.            0.         1208.20458731]
0 8 [-1 -1 -1  6] [   0.            0.            0.         1208.02700567]
0 10 [-1 -1 -1  6] [   0.            0.            0.         1207.85270959]
0 12 [-1 -1 -1  6] [   0.            0.            0.         1207.68170049]
0 14 [-1 -1 -1  6] [   0.            0.            0.         1207.51397976]
0 16 [-1 -1 -1  6] [   0.            0.            0.         1207.34954878]
0 18 [-1 -1 -1  6] [   0.           0.           0.        1207.1884089]
0 20 [-1 -1 -1  6] [   0.            0.            0.         1207.03056142]
0 22 [-1 -1 -1  6] [   0.            0.            0.         1206.87600765]
0 24 [-1 -1 -1  6] [   0.            0.            0.         1206.72474884]
0 26 [-1

0 342 [ -1   6  -1 229] [   0.         1224.70288579    0.          984.30167152]
0 344 [ -1   6  -1 229] [   0.         1225.07635684    0.          982.40260419]
0 346 [ -1   6  -1 229] [   0.         1225.45297817    0.          980.50393823]
0 348 [ -1   6  -1 229] [   0.         1225.83274689    0.          978.60567597]
0 350 [ -1   6  -1 229] [   0.         1226.21566006    0.          976.70781978]
0 352 [ -1   6  -1 229] [   0.         1226.60171474    0.          974.81037201]
0 354 [ -1   6  -1 229] [   0.         1226.99090796    0.          972.91333506]
0 356 [ -1   6  -1 229] [   0.         1227.38323675    0.          971.01671134]
0 358 [ -1   6  -1 229] [   0.         1227.77869809    0.          969.12050327]
0 360 [ -1   6  -1 229] [   0.         1228.17728896    0.          967.2247133 ]
0 362 [ -1   6  -1 229] [   0.         1228.57900631    0.          965.32934389]
0 364 [ -1   6  -1 229] [   0.         1228.98384707    0.          963.43439753]
0 366 [ -1   6  

0 662 [ -1   6  -1 229] [   0.         1322.13638444    0.          687.76825055]
0 664 [ -1   6  -1 229] [   0.         1322.96619031    0.          685.98124072]
0 666 [ -1   6  -1 229] [   0.         1323.79849763    0.          684.19540979]
0 668 [ -1   6  -1 229] [   0.         1324.63330169    0.          682.41076702]
0 670 [ -1   6  -1 229] [   0.         1325.47059778    0.          680.62732174]
0 672 [ -1   6  -1 229] [   0.         1326.31038118    0.          678.84508341]
0 674 [ -1   6  -1 229] [   0.         1327.15264715    0.          677.06406154]
0 676 [ -1   6  -1 229] [   0.         1327.99739099    0.          675.28426577]
0 678 [ -1   6  -1 229] [   0.         1328.84460796    0.          673.50570581]
0 680 [ -1   6  -1 229] [   0.         1329.69429334    0.          671.72839148]
0 682 [ -1   6  -1 229] [   0.         1330.54644239    0.          669.95233269]
0 684 [ -1   6  -1 229] [   0.         1331.40105039    0.          668.17753946]
0 686 [ -1   6  

0 988 [ -1 159  -1 229] [   0.         1395.17889346    0.          422.49241512]
0 990 [ -1 159  -1 229] [   0.         1395.21621063    0.          421.1281717 ]
0 992 [ -1 159  -1 229] [   0.         1395.25639366    0.          419.76902358]
0 994 [ -1 159  -1 229] [   0.         1395.29944231    0.          418.41502043]
0 996 [ -1 159  -1 229] [   0.         1395.3453563     0.          417.06621235]
0 998 [ -1 159  -1 229] [   0.         1395.39413536    0.          415.72264991]
0 1000 [ -1 159  -1 229] [   0.         1395.44577919    0.          414.38438413]
0 1002 [ -1 159  -1 229] [   0.         1395.50028746    0.          413.05146649]
0 1004 [ -1 159  -1 229] [   0.         1395.55765984    0.          411.72394894]
0 1006 [ -1 159  -1 229] [   0.         1395.61789598    0.          410.40188388]
0 1008 [ -1 159  -1 229] [   0.         1395.68099551    0.          409.08532418]
0 1010 [ -1 168  -1 229] [   0.         1369.52186116    0.          407.77432315]
0 1012 [ -

...

0 2302 [ -1 444  -1 515] [   0.          398.88027013    0.         1570.25367328]
0 2304 [ -1 444  -1 515] [   0.          400.2530373     0.         1570.15442982]
0 2306 [ -1 444  -1 515] [   0.          401.63107179    0.         1570.05772775]
0 2308 [ -1 444  -1 515] [   0.          403.01431959    0.         1569.96356757]
0 2310 [ -1 444  -1 515] [   0.          404.40272718    0.         1569.87194972]
0 2312 [ -1 444  -1 515] [   0.          405.79624163    0.         1569.78287464]
0 2314 [ -1 444  -1 515] [   0.          407.19481048    0.         1569.69634278]
0 2316 [ -1 444  -1 515] [   0.          408.59838184    0.         1569.61235455]
0 2318 [ -1 444  -1 515] [   0.          410.00690434    0.         1569.53091035]
0 2320 [ -1 444  -1 515] [   0.          411.42032712    0.         1569.4520106 ]
0 2322 [ -1 444  -1 515] [   0.          412.83859986    0.         1569.37565566]
0 2324 [ -1 444  -1 515] [   0.          414.26167273    0.         1569.30184591]
0 23

0 2610 [ -1 444  -1 544] [   0.          649.84241184    0.         1912.44077077]
0 2612 [ -1 444  -1 544] [   0.          651.63193921    0.         1912.42975974]
0 2614 [ -1 444  -1 544] [   0.          653.42268721    0.         1912.42084024]
0 2616 [ -1 444  -1 544] [   0.          655.21464584    0.         1912.41401229]
0 2618 [ -1 444  -1 544] [   0.          657.00780519    0.         1912.40927593]
0 2620 [ -1 444  -1 544] [   0.          658.80215546    0.         1912.40663115]
0 2622 [ -1 444  -1 547] [   0.          660.59768695    0.         1978.230321  ]
0 2624 [ -1 444  -1 547] [   0.          662.39439005    0.         1978.17650758]
0 2626 [ -1 444  -1 547] [   0.          664.19225525    0.         1978.12471481]
0 2628 [ -1 444  -1 547] [   0.          665.99127314    0.         1978.07494286]
0 2630 [ -1 444  -1 547] [   0.          667.7914344     0.         1978.02719188]
0 2632 [ -1 444  -1 547] [   0.          669.59272982    0.         1977.981462  ]
0 26

/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/Users/busko/Projects/software/miniconda3_2/envs/raw/lib/python3.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


2 106 [-1  2 -1  6] [   0.         2763.20162696    0.         1201.3721453 ]
2 108 [-1  2 -1  6] [   0.         2763.2288458     0.         1201.36004309]
2 110 [-1  2 -1  6] [   0.         2763.25751194    0.         1201.35127035]
2 112 [-1  2 -1  6] [   0.         2763.28762533    0.         1201.34582715]
2 114 [-1  2 -1  6] [   0.         2763.31918593    0.         1201.34371353]
2 116 [ -1   6  -1 229] [   0.         1201.34492951    0.         1200.49662298]
2 118 [ -1   6  -1 229] [   0.         1201.34947508    0.         1198.56298872]
2 120 [ -1   6  -1 229] [   0.         1201.35735022    0.         1196.62957263]
2 122 [ -1   6  -1 229] [   0.         1201.36855483    0.         1194.69637576]
2 124 [ -1   6  -1 229] [   0.         1201.38308885    0.         1192.76339918]
2 126 [ -1   6  -1 229] [   0.         1201.40095214    0.         1190.83064396]
2 128 [ -1   6  -1 229] [   0.         1201.42214456    0.         1188.89811117]
2 130 [ -1   6  -1 229] [   0.      

KeyboardInterrupt: 